In [118]:
from konlpy.tag import Okt
import re
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from itertools import zip_longest
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from tqdm import tqdm
import pickle
import glob
from datetime import datetime, timedelta
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis


pyLDAvis.enable_notebook()


In [116]:
def load_stopwords(file_path):
    """
    file_path 경로에서 불용어를 불러오는 함수
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        stopwords = f.read().splitlines()
    return stopwords


def preprocess_text(text, stopwords=None):
    """
    Okt를 사용한 텍스트 전처리 함수
    - 불용어 제거
    - 명사 추출
    """
    okt = Okt()
    # 정규 표현식을 사용한 숫자/특수문자 제거
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    nouns = okt.nouns(text)

    if stopwords:
        nouns = [word for word in nouns if word not in stopwords]
    
    # 길이가 1인 단어들 제거
    nouns = [word for word in nouns if len(word) > 1]

    processed_text = ' '.join(nouns)

    return processed_text


def load_and_merge_section_data(section_number, hours_back=9):
    """
    섹션 번호에 해당하는 모든 pkl 파일을 불러와서 기사 본문을 개별 문서로 병합하는 함수
    현재 시간에서 -hours_back 시간 전까지의 파일만 병합
    """
    current_time = datetime.now()
    start_time = current_time - timedelta(hours=hours_back)

    valid_hours = []
    for hour in range(hours_back + 1):  # 0부터 hours_back까지의 시간을 계산
        valid_hour = (start_time + timedelta(hours=hour)).strftime('%H')
        valid_hours.append(valid_hour)

    file_patterns = [f"./data/{section_number}/*_{hour}.pkl" for hour in valid_hours]

    merged_content = []
    for file_pattern in tqdm(file_patterns, desc=f"섹션 번호: {section_number} 파일"):
        file_list = glob.glob(file_pattern)

        for file_path in file_list:
            print(file_path)
            with open(file_path, 'rb') as file:
                data = pickle.load(file)
                for article in data:
                    merged_content.append(article['content'])

    print("merge len: " + str(len(merged_content)))

    return merged_content  # 각각의 기사를 리스트 형태로 반환

def visualize_lda_model(lda_model, corpus, dictionary):
    vis = gensimvis.prepare(lda_model, corpus, dictionary)
    return vis

# def process_sections(section_numbers, stopwords_file_path, hours_back=9):
#     """
#     주어진 섹션 번호 리스트에 대해 각 섹션별로 데이터를 병합하고 전처리하는 함수
#     """
#     # 불용어 로드
#     print("불용어를 불러오는 중입니다...")
#     stopwords = load_stopwords(stopwords_file_path)

#     processed_documents = []

#     for section_number in tqdm(section_numbers, desc="섹션 처리중"):
#         # 데이터 병합 (각 기사별로 처리)
#         merged_content_list = load_and_merge_section_data(section_number, hours_back)

#          # 각 기사를 개별 문서로 전처리
#         for content in merged_content_list:
#             processed_text = preprocess_text(content, stopwords)
#             processed_documents.append()
#             processed_documents.append(processed_text.split())  # 리스트 형태로 추가

#     return processed_documents  # 문서 리스트 반환

In [119]:
# def prepare_data_for_lda(doc):
#     """
#     LDA를 위한 데이터를 준비하는 함수
#     - doc: 전처리된 텍스트 데이터 (단일 섹션의 텍스트)
#     """
#     # 전처리된 문서를 리스트로 변환
#     text = doc.split()

#     # 딕셔너리 생성
#     dictionary = corpora.Dictionary([text])

#     # 코퍼스 생성(단어의 빈도수로 변환된 텍스트 데이터)
#     corpus = [dictionary.doc2bow(text)]

#     return dictionary, corpus

In [115]:
section_numbers = [100, 101, 102, 103, 104, 105]
stopwords_file_path = './data/korean_stopwords.txt'


section_documents = []

for section_number in section_numbers:
    stopwords = load_stopwords(stopwords_file_path)

    processed_documents = []

    merged_content_list = load_and_merge_section_data(section_number, hours_back=9)

    for content in merged_content_list:
        processed_text = preprocess_text(content, stopwords)
        processed_documents.append(processed_text.split())

    section_documents.append(processed_documents)
    

print(len(section_documents))



# processed_docs = process_sections(section_numbers, stopwords_file_path)


섹션 번호: 100 파일: 100%|██████████| 10/10 [00:00<00:00, 5010.52it/s]


./data/100\2024-09-24_11.pkl
merge len: 171


섹션 번호: 101 파일: 100%|██████████| 10/10 [00:00<00:00, 1219.27it/s]


./data/101\2024-09-24_11.pkl
merge len: 720


섹션 번호: 102 파일: 100%|██████████| 10/10 [00:00<00:00, 1238.61it/s]


...

./data/103\2024-09-24_11.pkl
merge len: 98


섹션 번호: 104 파일: 100%|██████████| 10/10 [00:00<00:00, 19645.45it/s]


./data/104\2024-09-24_11.pkl
merge len: 166


섹션 번호: 105 파일: 100%|██████████| 10/10 [00:00<00:00, 1139.82it/s]


./data/105\2024-09-24_11.pkl
merge len: 83
6


In [121]:
# 6개의 섹션에 대해 토픽 모델링 수행
for i in range(len(section_numbers)):
    dictionary = corpora.Dictionary(section_documents[i])
    corpus = [dictionary.doc2bow(text) for text in section_documents[i]]

    NUM_TOPICS = 3

    lda_model = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    topics = lda_model.print_topics(num_words=10)
    print(str(i+1 * 100) + "섹션의 토픽들")
    for topic in topics:
        print(topic)

    # 시각화 실행
    visualization = visualize_lda_model(lda_model, corpus, dictionary)
    pyLDAvis.save_html(visualization, f'lda_vis_{i+1*100}.html')

    

100섹션의 토픽들
(0, '0.013*"대통령" + 0.012*"민주당" + 0.011*"의원" + 0.010*"대표" + 0.009*"여사" + 0.009*"국민" + 0.007*"후보" + 0.007*"공천" + 0.007*"국회" + 0.007*"김건희"')
(1, '0.021*"대표" + 0.018*"독대" + 0.017*"대통령" + 0.012*"국민" + 0.011*"대통령실" + 0.008*"만찬" + 0.008*"금투세" + 0.008*"의원" + 0.007*"요청" + 0.007*"북한"')
(2, '0.015*"지금" + 0.013*"대표" + 0.012*"대통령" + 0.011*"통일" + 0.010*"얘기" + 0.009*"북한" + 0.007*"주장" + 0.007*"국민" + 0.007*"대해" + 0.006*"의원"')
101섹션의 토픽들
(0, '0.005*"사업" + 0.005*"개발" + 0.004*"가구" + 0.004*"예정" + 0.004*"제공" + 0.004*"미국" + 0.004*"분양" + 0.003*"브랜드" + 0.003*"투자" + 0.003*"서울"')
(1, '0.008*"금리" + 0.008*"시장" + 0.007*"미국" + 0.006*"가격" + 0.006*"증가" + 0.005*"지수" + 0.005*"인하" + 0.005*"주가" + 0.005*"상승" + 0.005*"투자"')
(2, '0.012*"고려아연" + 0.010*"기업" + 0.007*"투자" + 0.006*"기술" + 0.006*"경영" + 0.006*"사업" + 0.006*"회장" + 0.005*"파트너" + 0.005*"반도체" + 0.004*"회사"')
102섹션의 토픽들
(0, '0.006*"병원" + 0.006*"서울" + 0.005*"지역" + 0.004*"의원" + 0.004*"환자" + 0.004*"지난" + 0.003*"올해" + 0.003*"기자" + 0.003*"의료" + 0.003*"사업"')
(1, '0.00

In [120]:
# NUM_TOPICS = 6

# lda_model = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# topics = lda_model.print_topics(num_words=10)
# for topic in topics:
#     print(topic)
